In [1]:
from functions import get_access_token, sign_jwt
import requests
import json
import pandas as pd
from sqlalchemy import create_engine 
import datetime as dt
import hashlib
import urllib
from config import prod_params

In [2]:
SECRET_KEY = open('private.key', 'r').read()

In [3]:
jwt = sign_jwt(SECRET_KEY)
access_token = get_access_token(jwt)
headers = {'Authorization': "Bearer " + access_token}

In [5]:
### list of certificates 
certification_url = 'https://mccowngordon.docebosaas.com/learn/v1/certification'
certification_request = requests.get(url=certification_url, headers=headers)
certification_json = json.loads(certification_request.text)
certification_responses=list()
i=1
while certification_json['data']['has_more_data']== True:
    certification_url = f'https://mccowngordon.docebosaas.com/learn/v1/certification?page={i}'
    certification_request = requests.get(url=certification_url, headers=headers)
    certification_json = json.loads(certification_request.text)
    certification_json_data = certification_json['data']['items']
    certification_responses.append(certification_json_data)
    i+=1


In [6]:
certificationsDF = pd.concat([pd.json_normalize(i)for i in certification_responses])

In [7]:
#Gets all Courses
courses_url = 'https://mccowngordon.docebosaas.com/learn/v1/courses'
courses_request = requests.get(url=courses_url, headers=headers)
courses_json = json.loads(courses_request.text)
courses_responses=list()
i=1
while courses_json['data']['has_more_data']== True:
    courses_url = f'https://mccowngordon.docebosaas.com/learn/v1/courses?page={i}'
    courses_request = requests.get(url=courses_url, headers=headers)
    courses_json = json.loads(courses_request.text)
    courses_json_data = courses_json['data']['items']
    courses_responses.append(courses_json_data)
    i+=1



In [ ]:
courses_url = 'https://mccowngordon.docebosaas.com/learn/v1/courses'
courses_request = requests.get(url=courses_url, headers=headers)
courses_json = json.loads(courses_request.text)
courses_json

In [8]:
coursesDF = pd.concat([pd.json_normalize(i)for i in courses_responses])

In [9]:
#Gets all the external training
Externaltraining_url = 'https://mccowngordon.docebosaas.com/learn/v1/external_training?page=1'
Externaltraining_request = requests.get(url=Externaltraining_url, headers=headers)
Externaltraining_json = json.loads(Externaltraining_request.text)
Externaltraining_responses=list()
i=1
while Externaltraining_json['data']['has_more_data']== True or i==1:
    Externaltraining_url = f'https://mccowngordon.docebosaas.com/learn/v1/external_training?page={i}'
    Externaltraining_request = requests.get(url=Externaltraining_url, headers=headers)
    Externaltraining_json = json.loads(Externaltraining_request.text)
    Externaltraining_json_data = Externaltraining_json['data']['items']
    Externaltraining_responses.append(Externaltraining_json_data)
    i+=1

In [ ]:
ExternaltrainingDF = pd.concat([pd.json_normalize(i)for i in Externaltraining_responses])
ExternaltrainingDF

In [ ]:
#Shows all enrollments, including completed. Takes over 12 minutes to load all the data. 
enrollments_url = 'https://mccowngordon.docebosaas.com/learn/v1/enrollments?page_size=500'
enrollments_request = requests.get(url=enrollments_url, headers=headers)
enrollments_json = json.loads(enrollments_request.text)
enrollments_responses=list()
i=1
while enrollments_json['data']['has_more_data']== True or i==1:
    enrollments_url = f'https://mccowngordon.docebosaas.com/learn/v1/enrollments?page={i}&page_size=500'
    enrollments_request = requests.get(url=enrollments_url, headers=headers)
    enrollments_json = json.loads(enrollments_request.text)
    enrollments_json_data = enrollments_json['data']['items']
    enrollments_responses.append(enrollments_json_data)
    i+=1
    print(i)




In [ ]:
#Get's all user information
users_url = 'https://mccowngordon.docebosaas.com/manage/v1/user'
users_request = requests.get(url=users_url, headers=headers)
users_json = json.loads(users_request.text)
users_responses=list()
i=1
while users_json['data']['has_more_data']== True or i==1:
    users_url = f'https://mccowngordon.docebosaas.com/manage/v1/user?page={i}'
    users_request = requests.get(url=users_url, headers=headers)
    users_json = json.loads(users_request.text)
    users_json_data = users_json['data']['items']
    users_responses.append(users_json_data)
    i+=1
    print(i)


In [5]:
usersDF = pd.concat([pd.json_normalize(i)for i in users_responses])

In [27]:
enrollmentsDF = pd.concat([pd.json_normalize(i)for i in enrollments_responses])
df=enrollmentsDF


In [15]:
 
prod_conn_params = urllib.parse.quote_plus(prod_params) 
prod_conn_str = "mssql+pyodbc:///?odbc_connect={}".format(prod_conn_params) 
engine_azure = create_engine(prod_conn_str, fast_executemany=True) 

def add_metadata(df, source_id='ENTER SOURCE ID'): 
    df["stgLoadDateTime"] = dt.datetime.now() 
    load_time_stamp = df["stgLoadDateTime"][0] 
    str2hash = load_time_stamp.strftime('%Y-%m-%d %X') 
    df["stgSourceID"] = source_id 
    result = hashlib.md5(str(str2hash).encode()) 
    df["stgPipelineRunID"] = result.hexdigest()   
    return df 

In [ ]:
# enrollmentDF_metadata = add_metadata(enrollmentsDF, source_id=21)
# externalTrainingDF_metadata = add_metadata(ExternaltrainingDF, source_id=21)
# coursesDF_metadata= add_metadata(coursesDF, source_id=21)
# certificationsDF_metadata= add_metadata(certificationsDF, source_id=21)
# UsersDF_metadata= add_metadata(usersDF, source_id=21)


enrollmentsDF

In [31]:
enrollmentsDF['actions'] = enrollmentsDF['actions'].astype(str)

In [ ]:
enrollmentsDF.to_sql( 
    schema="docebo", 
    name="enrollment", 
    con=engine_azure, 
    index=False, 
    if_exists="replace",   
) 

In [17]:
ExternaltrainingDF['additional_fields'] = ExternaltrainingDF['additional_fields'].astype(str)

In [ ]:
ExternaltrainingDF.to_sql( 
    schema="docebo", 
    name="external_training", 
    con=engine_azure, 
    index=False, 
    if_exists="replace",   
) 

In [ ]:
coursesDF.to_sql( 
    schema="docebo", 
    name="courses", 
    con=engine_azure, 
    index=False, 
    if_exists="replace",   
) 

In [ ]:
certificationsDF.to_sql( 
    schema="docebo", 
    name="certifications", 
    con=engine_azure, 
    index=False, 
    if_exists="replace",   
) 

In [39]:
usersDF['actions'] = usersDF['actions'].astype(str)
usersDF['managers'] = usersDF['managers'].astype(str)
usersDF['multidomains'] = usersDF['multidomains'].astype(str)



In [ ]:
usersDF.to_sql( 
    schema="docebo", 
    name="users", 
    con=engine_azure, 
    index=False, 
    if_exists="replace",   
) 

In [ ]:
usersX_url = 'https://mccowngordon.docebosaas.com/manage/v1/user/14135'
usersX_request = requests.get(url=usersX_url, headers=headers)
usersX_json = json.loads(usersX_request.text)
usersX_json

In [ ]:
#Create a dictionary linking every user_id with their branch ID
BranchUsersDict={}
FailedUsers=list()
for i in usersDF['user_id']:
    print(i)
    usersX_url = f'https://mccowngordon.docebosaas.com/manage/v1/user/{i}'
    usersX_request = requests.get(url=usersX_url, headers=headers)
    usersX_json = json.loads(usersX_request.text)
    try:
        BranchUsersDict.update({i:usersX_json['data']['branches'][0]['id']})
    except: 
        FailedUsers.append(i)

In [ ]:
BranchUsersDict

In [ ]:
Users_BranchesDF = pd.DataFrame(list(BranchUsersDict.items()))
Users_BranchesDF.rename(columns={0:'user_id', 1:'branch_id'}, inplace=True)
Users_BranchesDF

In [29]:
Users_BranchesDF.to_sql( 
    schema="docebo", 
    name="users_branch", 
    con=engine_azure, 
    index=False, 
    if_exists="replace",   
) 

-1

In [ ]:
#Not Category, this applies the each course instead of each user.

# category_url = 'https://mccowngordon.docebosaas.com/learn/v1/categories'
# category_request = requests.get(url=category_url, headers=headers)
# category_json = json.loads(category_request.text)
# category_json

In [ ]:
### Probably Not Needed

# department_url = 'https://mccowngordon.docebosaas.com/report/v1/branch_dashboard_enrollment/0/view'
# department_request = requests.get(url=department_url, headers=headers)
# department_json = json.loads(department_request.text)
# department_json

In [7]:
#Get's all branches and their children, only departments (id=2) and McCownGordon (id=0) needs to be called to get all branches

branchlist= list()
for i in range(0,100):
    branch_url = f'https://mccowngordon.docebosaas.com/report/v1/branch_dashboard_enrollment/{i}/view'
    branch_request = requests.get(url=branch_url, headers=headers)
    branch_json = json.loads(branch_request.text)
    if branch_request.status_code == 200:
        branchlist.append(branch_json)
        

In [ ]:
branchlistMGC=branchlist[0]['data']['items']
branchlistMGC

In [ ]:
branchMGCDF = pd.DataFrame.from_dict(branchlistMGC)
branchMGCDF

In [ ]:
branchlistDepartments=branchlist[1]['data']['items']
branchlistDepartments

In [ ]:
branchDepartmentsDF = pd.DataFrame.from_dict(branchlistDepartments)
branchDepartmentsDF

In [ ]:
BranchesDF= pd.concat([branchMGCDF, branchDepartmentsDF])
BranchesDF

In [16]:
BranchesDF.to_sql( 
    schema="docebo", 
    name="Branches", 
    con=engine_azure, 
    index=False, 
    if_exists="replace",   
) 

-1

In [ ]:
group_url = 'https://mccowngordon.docebosaas.com/manage/v1/group'
group_request = requests.get(url=group_url, headers=headers)
group_json = json.loads(group_request.text)
group_json

In [ ]:
#Use Group end point to get the deparment name and ID. use this endpoint to get members of the group by ID number
groupMembers_url = 'https://mccowngordon.docebosaas.com/manage/v1/group/13052/members'
groupMembers_request = requests.get(url=groupMembers_url, headers=headers)
groupMembers_json = json.loads(groupMembers_request.text)
groupMembers_json